In [15]:
%%configure -f
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1665741913293_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1665741913293_0002,pyspark,idle,Link,Link,None,✔


# Test no 2 - Users

## Data prep 

In [3]:
# Create a DataFrame
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([
    StructField('dateTimeOccurred', StringType(), False),
    StructField('data', StructType([ 
        StructField('id', StringType(), False),
        StructField('auth_id', StringType(), True),
        StructField('first_name', StringType(), True),
        StructField('last_name', StringType(), True),
        StructField('phone', StringType(), True),
        StructField('primary_email', StringType(), True),
        StructField('country', StringType(), True),
        StructField('address', StringType(), True),
        StructField('zipcode', StringType(), True),
        StructField('source', StringType(), True),
        StructField('created_at', StringType(), True),
        StructField('updated_at', StringType(), True),
        StructField('deleted_at', StringType(), True)
    ]), True),
StructField('eventSource', StringType(), False),
StructField('eventType', StringType(), False),
StructField('eventId', StringType(), True) 
    ])
inputDF = spark.createDataFrame(
    [
        ("2022-09-22T08:00:00.869Z", ("61998a31-cbd8-47de-9b2b-554b0377a000", "7d723ebd-f9cb-4b78-be1f-b6d8021a4187", "Jess", "", "+0048272699519", "jacacio.d@gmail.com", "", "", "", "atfarm", "1519906269690", "1519906280718", None), "ffdp_central_store.farm_profile.users", "Insert", "2022-09-22-08-00-00-000000"),
        ("2022-09-22T08:01:00.869Z", ("26eff161-dbf9-45e3-bc9d-b311224858c2", "cf87173b-aa25-4954-91df-45a8063dc50d", "Johannes Boyne", "", "+0048611016809", "johannesboyne+yara-staging-1@gmail.com", "", "", "", "atfarm", "1519906269690", "1520077642240", None), "ffdp_central_store.farm_profile.users", "Insert", "2022-09-22-08-01-00-000001"),
        ("2022-09-22T08:04:00.869Z", ("c2b86d7f-95a3-4f65-9863-d5338e99bcf0", "8ad9e19b-b8d1-452b-a689-8469893fddff", "Max Mustermann", "", "+0048664268672", "johannesboyne+yara123@gmail.com", "", "", "", "atfarm", "1520432203566", "1520432203566", None), "ffdp_central_store.farm_profile.users", "Insert", "2022-09-22-08-04-00-000004")
        
    ],
    schema
)
inputDF.printSchema()
inputDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dateTimeOccurred: string (nullable = false)
 |-- data: struct (nullable = true)
 |    |-- id: string (nullable = false)
 |    |-- auth_id: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- phone: string (nullable = true)
 |    |-- primary_email: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- updated_at: string (nullable = true)
 |    |-- deleted_at: string (nullable = true)
 |-- eventSource: string (nullable = false)
 |-- eventType: string (nullable = false)
 |-- eventId: string (nullable = true)

+--------------------+--------------------+--------------------+---------+--------------------+
|    dateTimeOccurred|                data|         eventSource|eventType|             eventId|
+------

## CDC via Glue Checks

In [2]:
hudiOptions = {
'hoodie.table.name': 'cdc_users',
'hoodie.datasource.write.recordkey.field': 'id',
'hoodie.datasource.write.partitionpath.field': 'created_at',
'hoodie.datasource.write.precombine.field': 'dateTimeOccurred',
'hoodie.datasource.hive_sync.enable': 'true',
'hoodie.datasource.hive_sync.table': 'cdc_users',
'hoodie.datasource.hive_sync.partition_fields': 'created_at',
'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1665741913293_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load('s3://aws-emr-resources-479055760150-eu-central-1/cdc_users' + '/*/*') \
    .select("id", "first_name", "primary_email", "created_at", "updated_at", "eventType")
# .filter("eventType = 'Update'") \
    
snapshotQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+-------------+-------------+---------+
|                  id|    first_name|       primary_email|   created_at|   updated_at|eventType|
+--------------------+--------------+--------------------+-------------+-------------+---------+
|7b285a3e-42c9-411...|Johannes Boyne|johannesboyne+yar...|1520433144493|1520433144494|   Insert|
|c2b86d7f-95a3-4f6...|Max Mustermann|johannesboyne+yar...|1520432203566|1520432203566|   Insert|
|61998a31-cbd8-47d...|          Jess| jacacio.d@gmail.com|1519906269690|1519906280718|   Insert|
+--------------------+--------------+--------------------+-------------+-------------+---------+

In [13]:
rawDF = spark.read.parquet(f"s3://aws-emr-resources-479055760150-eu-central-1/cdc_users/*/*.parquet")
# rawDF.show(rawDF.count(), truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
last_commit_time = rawDF.sort(rawDF._hoodie_commit_time.desc()).select("_hoodie_commit_time").limit(1).collect()[0]._hoodie_commit_time
print(last_commit_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20221013101630531

In [10]:
readOptions = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': '0',
  'hoodie.datasource.read.end.instanttime': last_commit_time,
}

incQueryDF = spark.read \
    .format('org.apache.hudi') \
    .options(**readOptions) \
    .load(f"s3://aws-emr-resources-479055760150-eu-central-1/cdc_users") \
    .select("_hoodie_commit_time", "id", "first_name", "primary_email", "created_at", "updated_at", "eventType")
    
incQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|_hoodie_commit_time|                  id|    first_name|       primary_email|   created_at|   updated_at|eventType|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|  20221013100436088|26eff161-dbf9-45e...|Johannes Boyne|johannesboyne+yar...|1520077642240|1520077642240|   Update|
|  20221013101630531|7b285a3e-42c9-411...|Johannes Boyne|johannesboyne+yar...|1520433144493|1520433144494|   Insert|
|  20221013101630531|c2b86d7f-95a3-4f6...|Max Mustermann|johannesboyne+yar...|1520432203566|1520432203566|   Insert|
|  20221013101630531|61998a31-cbd8-47d...|          Jess| jacacio.d@gmail.com|1519906269690|1519906280718|   Insert|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+

In [11]:
readOptions = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': '0',
  'hoodie.datasource.read.end.instanttime': '20221013100436088', #last_commit_time,
}

incQueryDF = spark.read \
    .format('org.apache.hudi') \
    .options(**readOptions) \
    .load(f"s3://aws-emr-resources-479055760150-eu-central-1/cdc_users") \
    .select("_hoodie_commit_time", "id", "first_name", "primary_email", "created_at", "updated_at", "eventType")
    
incQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|_hoodie_commit_time|                  id|    first_name|       primary_email|   created_at|   updated_at|eventType|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|  20221013100436088|26eff161-dbf9-45e...|Johannes Boyne|johannesboyne+yar...|1520077642240|1520077642240|   Update|
|  20221013100436088|7b285a3e-42c9-411...|Johannes Boyne|johannesboyne+yar...|1520433144493|1520433144494|   Insert|
|  20221013100436088|c2b86d7f-95a3-4f6...|Max Mustermann|johannesboyne+yar...|1520432203566|1520432203566|   Insert|
|  20221013100436088|61998a31-cbd8-47d...|          Jess| jacacio.d@gmail.com|1519906269690|1519906280718|   Insert|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+

In [12]:
first_commit_time = rawDF.sort(rawDF._hoodie_commit_time.asc()).select("_hoodie_commit_time").limit(1).collect()[0]._hoodie_commit_time

readOptions = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': '0',
  'hoodie.datasource.read.end.instanttime': 'first_commit_time',
}

incQueryDF = spark.read \
    .format('org.apache.hudi') \
    .options(**readOptions) \
    .load(f"s3://aws-emr-resources-479055760150-eu-central-1/cdc_users") \
    .select("_hoodie_commit_time", "id", "first_name", "primary_email", "created_at", "updated_at", "eventType")
    
incQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|_hoodie_commit_time|                  id|    first_name|       primary_email|   created_at|   updated_at|eventType|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+
|  20221013101630531|7b285a3e-42c9-411...|Johannes Boyne|johannesboyne+yar...|1520433144493|1520433144494|   Insert|
|  20221013101630531|c2b86d7f-95a3-4f6...|Max Mustermann|johannesboyne+yar...|1520432203566|1520432203566|   Insert|
|  20221013101630531|61998a31-cbd8-47d...|          Jess| jacacio.d@gmail.com|1519906269690|1519906280718|   Insert|
+-------------------+--------------------+--------------+--------------------+-------------+-------------+---------+

## Test with Users_Table

In [3]:
# Specify common DataSourceWriteOptions in the single hudiOptions variable
hudiOptions = {
'hoodie.table.name': 'users_table',
'hoodie.datasource.write.recordkey.field': 'data.id',
'hoodie.datasource.write.partitionpath.field': 'data.created_at',
'hoodie.datasource.write.precombine.field': 'dateTimeOccurred',
'hoodie.datasource.hive_sync.enable': 'true',
'hoodie.datasource.hive_sync.table': 'users_table',
'hoodie.datasource.hive_sync.partition_fields': 'created_at',
'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_BUCKET_NAME = "aws-emr-resources-479055760150-eu-central-1"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Insert

In [14]:
# Write a DataFrame as a Hudi dataset
inputDF.write \
.format('org.apache.hudi') \
.option('hoodie.datasource.write.operation', 'insert') \
.options(**hudiOptions) \
.mode('overwrite') \
.save(f"s3://{S3_BUCKET_NAME}/users/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load('s3://aws-emr-resources-479055760150-eu-central-1/users' + '/*/*') \
    .select("data.id", "data.first_name", "data.primary_email", "data.created_at", "data.updated_at", "eventType")
    
snapshotQueryDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+-------------+-------------+---------+
|                  id|    first_name|       primary_email|   created_at|   updated_at|eventType|
+--------------------+--------------+--------------------+-------------+-------------+---------+
|c2b86d7f-95a3-4f6...|Max Mustermann|johannesboyne+yar...|1520432203566|1520432203566|   Insert|
|61998a31-cbd8-47d...|          Jess| jacacio.d@gmail.com|1519906269690|1519906280718|   Insert|
|26eff161-dbf9-45e...|Johannes Boyne|johannesboyne+yar...|1519906269690|1520077642240|   Insert|
+--------------------+--------------+--------------------+-------------+-------------+---------+

In [9]:
%env S3_BUCKET_NAME=aws-emr-resources-479055760150-eu-central-1

env: S3_BUCKET_NAME=aws-emr-resources-479055760150-eu-central-1


In [22]:
%%sh

aws s3 ls s3://${S3_BUCKET_NAME}/users/

                           PRE .hoodie/
                           PRE 1519906269690/
2022-10-06 06:06:14          0 .hoodie_$folder$
2022-10-06 06:06:36          0 1519906269690_$folder$


## Update

In [16]:
from pyspark.sql.functions import lit

# Create a new DataFrame from the first row of inputDF with a different creation_date value
updateDF = inputDF.filter("data.id = '61998a31-cbd8-47de-9b2b-554b0377a000'").withColumn("eventType", lit("Update"))

updateDF.select("data.id", "data.first_name", "data.primary_email", "data.created_at", "data.updated_at", "eventType").show()

updateDF.write.format("org.apache.hudi").option(
    "hoodie.datasource.write.operation", "upsert"
).options(**hudiOptions).mode("append").save(f"s3://{S3_BUCKET_NAME}/users/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-------------------+-------------+-------------+---------+
|                  id|first_name|      primary_email|   created_at|   updated_at|eventType|
+--------------------+----------+-------------------+-------------+-------------+---------+
|61998a31-cbd8-47d...|      Jess|jacacio.d@gmail.com|1519906269690|1519906280718|   Update|
+--------------------+----------+-------------------+-------------+-------------+---------+

In [17]:
snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load(f"s3://{S3_BUCKET_NAME}/users/") \
    .select("data.id", "data.first_name", "data.created_at", "data.updated_at", "eventType") \
    .sort('dateTimeOccurred', 'data.id')
    
snapshotQueryDF.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+--------------+-------------+-------------+---------+
|id                                  |first_name    |created_at   |updated_at   |eventType|
+------------------------------------+--------------+-------------+-------------+---------+
|61998a31-cbd8-47de-9b2b-554b0377a000|Jess          |1519906269690|1519906280718|Update   |
|26eff161-dbf9-45e3-bc9d-b311224858c2|Johannes Boyne|1519906269690|1520077642240|Insert   |
|c2b86d7f-95a3-4f65-9863-d5338e99bcf0|Max Mustermann|1520432203566|1520432203566|Insert   |
+------------------------------------+--------------+-------------+-------------+---------+

## Delete
Applying hard delete.

Using **"updateDF"** created above!

In [18]:
updateDF.write.format("org.apache.hudi").option(
    "hoodie.datasource.write.operation", "upsert"
).option(
    "hoodie.datasource.write.payload.class",
    "org.apache.hudi.common.model.EmptyHoodieRecordPayload",
).options(
    **hudiOptions
).mode(
    "append"
).save(
    f"s3://{S3_BUCKET_NAME}/users/"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load(f"s3://{S3_BUCKET_NAME}/users") \
    .select("data.id", "data.first_name", "data.created_at", "data.updated_at", "eventType") \
#     .withColumn("filename", regexp_replace(input_file_name(), S3_BUCKET_NAME, '<BUCKET>'))
    
snapshotQueryDF.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+--------------+-------------+-------------+---------+
|id                                  |first_name    |created_at   |updated_at   |eventType|
+------------------------------------+--------------+-------------+-------------+---------+
|26eff161-dbf9-45e3-bc9d-b311224858c2|Johannes Boyne|1519906269690|1520077642240|Insert   |
|c2b86d7f-95a3-4f65-9863-d5338e99bcf0|Max Mustermann|1520432203566|1520432203566|Insert   |
+------------------------------------+--------------+-------------+-------------+---------+